<a href="https://colab.research.google.com/github/annefrancine/Deep-Learning-Models-/blob/master/AutoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MovieLens dataset**

https://grouplens.org/datasets/movielens/

# Importing Packages

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
movies = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
users = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [6]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [0]:

ratings = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [8]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Train, test dataset

In [0]:
training_set = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/drive/My Drive/curso deep learning-udemy/Notebooks/7.BM/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [10]:
training_set.shape, test_set.shape

((79999, 4), (19999, 4))

# Number of users and films

In [0]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting data into a matrix with users in rows and movies in columns

In [0]:

def convert(data):
    new_data = [] 
    for id_users in range(1, nb_users + 1): 
        id_movies = data[:,1][data[:,0] == id_users] 
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Creating Torch Tensors

In [0]:

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the neural network architecture

In [0]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training neural network

In [15]:
#Treinando a RNA
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0. 
    for id_user in range(nb_users): 
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7714)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0216)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0197)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0186)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0174)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0170)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0157)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0155)
epoch: 26 loss: tensor(1.0155)
epoch: 27 loss: tensor(1.0151)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0129)
epoch: 30 loss: tensor(1.0115)
epoch: 31 loss: tensor(1.0102)
epoch: 32 loss: tensor(1.0082)
epoch: 33 loss: t

# Testing neural network


In [16]:

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input) 
        target.require_grad = False 
        output[target == 0] = 0 
        loss = criterion(output, target) 
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('Loss de test:: '+str(test_loss/s))

Loss de test:: tensor(0.9457)
